In [20]:
import csv
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np
import folium
from itertools import chain   #Functions creating iterators for efficient looping

In [3]:
conn = sqlite3.connect('patriGrenoble.db')  # Create DATABASE and CONNECTION
print('DataBase is connected')   # Checking that it is created

cur = conn.cursor()  # Create CURSOR
patri_Grenoble = pd.read_csv('/home/zahra/Simplon/Gitlab/projet_immo_folium/PATRIMOINE_VDG.csv')  # Uploading file.csv
                                              # pd_patri is a my DataFrame
patri_Grenoble

DataBase is connected


,Idtf,Titre,Thématiques,Périodes,Types,Adresse,Latitude,Longitude,Texte FR,Texte EN,Bibliographie,Site internet,Crédit,Légende
0,14,Échauguette de l'ancienne citadelle,"Histoire & Evolution de la ville, Ville fortif...",3ème période : 1590 / 1830,"Patrimoine Monumental (Militaire), Patrimoine ...","Quai Jongkind, Grenoble",45.194816,5.730774,"<h3>UN SYSTÈME DÉFENSIF, UNE NOUVELLE ENCEINTE...",NaN,NaN,NaN,NaN,NaN
1,20,"Place Victor-Hugo, cœur du nouveau centre-ville",Histoire & Evolution de la ville,5ème période : 1880 / 1925,Patrimoine Urbain (Place),"Place Victor Hugo, Grenoble",45.189111,5.724503,<p>UNE NOUVELLE URBANISATION ET SES ESPACES PU...,NaN,"- J-F. Parent, « Grenoble : deux siècles d'urb...",NaN,NaN,NaN
2,21,Enceinte romaine du 3e siècle,"Histoire & Evolution de la ville, Ville fortif...",1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Enceinte / Rempart),Géolocalisation incomplète (un vestige parmi d...,45.192529,5.731154,LES TRACES D'UNE VILLE-GALLO ROMAINE Première ...,NaN,Office du Tourisme : www.grenoble-tourisme.com,NaN,NaN,NaN
3,22,Bourg de Cularo,"Histoire & Evolution de la ville, Ville fortif...",1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Rue),"Place Grenette, Grenoble\r\nGéolocalisation in...",45.191052,5.727750,<p>LES TRACES D'UNE VILLE GALLO-ROMAINE Hormis...,NaN,"- I. Lazier et M. Vivier, « L'Isère en histoir...",NaN,NaN,NaN
4,23,Baptistère,Histoire & Evolution de la ville,1ére période : -43 / fin VIIe début VIIIe,Patrimoine Monumental (Religieux),"Musée de l'Ancien Évêché, 2 Rue Très Cloîtres,...",45.193059,5.732142,UN SITE ARCHÉOLOGIQUE MAJEUR Aux premiers temp...,NaN,"- A. Cayol-Gérin, V. Charbonnier, A. de Montjo...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,353,Dans les pas de Stendhal : l’hôtel de Lesdigui...,Art et culture,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","Passage du Palais de Justice, Grenoble\r\n",45.192750,5.727186,L'Hôtel de Lesdiguières et son jardin attenant...,NaN,NaN,NaN,Bibliothèque d'Etude et du Patrimoine et Ville...,NaN
246,356,Dans les pas de Stendhal : les collections de ...,"Art et culture, Tourisme",3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","12, boulevard Maréchal-Lyautey, Grenoble",45.185253,5.731065,Historique de la constitution des collections ...,NaN,Mille ans d’écrits : trésors de la bibliothèqu...,http://www.bm-grenoble.fr/596-stendhal.htm,Bibliothèque d'Etude et du Patrimoine et Ville...,NaN
247,357,Église Saint-Pierre-du-Rondeau,Art et culture,7ème période : 1968 / aube XXIe,Patrimoine Monumental (Religieux),"Eglise Saint-Pierre-du-Rondeau, Grenoble",45.169375,5.711883,B. Avezou et P. Blondeau (architectes) Suite à...,NaN,"- J. Pichard (dir.), « Les églises du diocèse ...",NaN,Directions des Affaires Culturelles et de l'Ur...,NaN
248,358,Protection du patrimoine bâti et des espaces p...,Histoire & Evolution de la ville,8ème période : XXIe,"Patrimoine Monumental, Patrimoine Paysager, Pa...",NaN,0.000000,0.000000,La politique patrimoniale de la Ville de Greno...,NaN,NaN,NaN,Directions des Affaires Culturelles et de l'Ur...,NaN


In [4]:
patri_Grenoble.describe()

,Idtf,Latitude,Longitude
count,250.000000,250.000000,250.000000
mean,191.476000,40.127136,5.086895
std,102.665174,14.279441,1.810244
min,14.000000,0.000000,0.000000
25%,100.250000,45.181650,5.721934
50%,183.500000,45.189931,5.729126
75%,284.750000,45.192971,5.733043
max,359.000000,45.339375,5.812808


In [5]:
patri_Grenoble.isnull().sum()

Idtf               0
Titre              0
Thématiques        0
Périodes           0
Types              0
Adresse           56
Latitude           0
Longitude          0
Texte FR           0
Texte EN         215
Bibliographie     56
Site internet    196
Crédit           125
Légende          249
dtype: int64

In [6]:
patri_Grenoble = patri_Grenoble.drop(['Légende', 'Texte EN', 'Site internet'], axis=1)  #axis:1 for column, axis:0 for row
patri_Grenoble

,Idtf,Titre,Thématiques,Périodes,Types,Adresse,Latitude,Longitude,Texte FR,Bibliographie,Crédit
0,14,Échauguette de l'ancienne citadelle,"Histoire & Evolution de la ville, Ville fortif...",3ème période : 1590 / 1830,"Patrimoine Monumental (Militaire), Patrimoine ...","Quai Jongkind, Grenoble",45.194816,5.730774,"<h3>UN SYSTÈME DÉFENSIF, UNE NOUVELLE ENCEINTE...",NaN,NaN
1,20,"Place Victor-Hugo, cœur du nouveau centre-ville",Histoire & Evolution de la ville,5ème période : 1880 / 1925,Patrimoine Urbain (Place),"Place Victor Hugo, Grenoble",45.189111,5.724503,<p>UNE NOUVELLE URBANISATION ET SES ESPACES PU...,"- J-F. Parent, « Grenoble : deux siècles d'urb...",NaN
2,21,Enceinte romaine du 3e siècle,"Histoire & Evolution de la ville, Ville fortif...",1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Enceinte / Rempart),Géolocalisation incomplète (un vestige parmi d...,45.192529,5.731154,LES TRACES D'UNE VILLE-GALLO ROMAINE Première ...,Office du Tourisme : www.grenoble-tourisme.com,NaN
3,22,Bourg de Cularo,"Histoire & Evolution de la ville, Ville fortif...",1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Rue),"Place Grenette, Grenoble\r\nGéolocalisation in...",45.191052,5.727750,<p>LES TRACES D'UNE VILLE GALLO-ROMAINE Hormis...,"- I. Lazier et M. Vivier, « L'Isère en histoir...",NaN
4,23,Baptistère,Histoire & Evolution de la ville,1ére période : -43 / fin VIIe début VIIIe,Patrimoine Monumental (Religieux),"Musée de l'Ancien Évêché, 2 Rue Très Cloîtres,...",45.193059,5.732142,UN SITE ARCHÉOLOGIQUE MAJEUR Aux premiers temp...,"- A. Cayol-Gérin, V. Charbonnier, A. de Montjo...",NaN
...,...,...,...,...,...,...,...,...,...,...,...
245,353,Dans les pas de Stendhal : l’hôtel de Lesdigui...,Art et culture,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","Passage du Palais de Justice, Grenoble\r\n",45.192750,5.727186,L'Hôtel de Lesdiguières et son jardin attenant...,NaN,Bibliothèque d'Etude et du Patrimoine et Ville...
246,356,Dans les pas de Stendhal : les collections de ...,"Art et culture, Tourisme",3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","12, boulevard Maréchal-Lyautey, Grenoble",45.185253,5.731065,Historique de la constitution des collections ...,Mille ans d’écrits : trésors de la bibliothèqu...,Bibliothèque d'Etude et du Patrimoine et Ville...
247,357,Église Saint-Pierre-du-Rondeau,Art et culture,7ème période : 1968 / aube XXIe,Patrimoine Monumental (Religieux),"Eglise Saint-Pierre-du-Rondeau, Grenoble",45.169375,5.711883,B. Avezou et P. Blondeau (architectes) Suite à...,"- J. Pichard (dir.), « Les églises du diocèse ...",Directions des Affaires Culturelles et de l'Ur...
248,358,Protection du patrimoine bâti et des espaces p...,Histoire & Evolution de la ville,8ème période : XXIe,"Patrimoine Monumental, Patrimoine Paysager, Pa...",NaN,0.000000,0.000000,La politique patrimoniale de la Ville de Greno...,NaN,Directions des Affaires Culturelles et de l'Ur...


In [7]:
### One way ###

# return list from series of comma-separated strings
def thematic(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = patri_Grenoble['Thématiques'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
new = pd.DataFrame({'Idtf': np.repeat(patri_Grenoble['Idtf'], lens),
                    'Titre': np.repeat(patri_Grenoble['Titre'], lens),
                    'Thématiques': thematic(patri_Grenoble['Thématiques']),
                    'Périodes': np.repeat(patri_Grenoble['Périodes'], lens),
                    'Types': np.repeat(patri_Grenoble['Types'], lens),
                    'Adresse': np.repeat(patri_Grenoble['Adresse'], lens),
                    'Latitude': np.repeat(patri_Grenoble['Latitude'], lens),
                    'Longitude': np.repeat(patri_Grenoble['Longitude'], lens)})

new

,Idtf,Titre,Thématiques,Périodes,Types,Adresse,Latitude,Longitude
0,14,Échauguette de l'ancienne citadelle,Histoire & Evolution de la ville,3ème période : 1590 / 1830,"Patrimoine Monumental (Militaire), Patrimoine ...","Quai Jongkind, Grenoble",45.194816,5.730774
0,14,Échauguette de l'ancienne citadelle,Ville fortifiée/de garnison/militaire,3ème période : 1590 / 1830,"Patrimoine Monumental (Militaire), Patrimoine ...","Quai Jongkind, Grenoble",45.194816,5.730774
1,20,"Place Victor-Hugo, cœur du nouveau centre-ville",Histoire & Evolution de la ville,5ème période : 1880 / 1925,Patrimoine Urbain (Place),"Place Victor Hugo, Grenoble",45.189111,5.724503
2,21,Enceinte romaine du 3e siècle,Histoire & Evolution de la ville,1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Enceinte / Rempart),Géolocalisation incomplète (un vestige parmi d...,45.192529,5.731154
2,21,Enceinte romaine du 3e siècle,Ville fortifiée/de garnison/militaire,1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Enceinte / Rempart),Géolocalisation incomplète (un vestige parmi d...,45.192529,5.731154
...,...,...,...,...,...,...,...,...
246,356,Dans les pas de Stendhal : les collections de ...,Art et culture,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","12, boulevard Maréchal-Lyautey, Grenoble",45.185253,5.731065
246,356,Dans les pas de Stendhal : les collections de ...,Tourisme,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","12, boulevard Maréchal-Lyautey, Grenoble",45.185253,5.731065
247,357,Église Saint-Pierre-du-Rondeau,Art et culture,7ème période : 1968 / aube XXIe,Patrimoine Monumental (Religieux),"Eglise Saint-Pierre-du-Rondeau, Grenoble",45.169375,5.711883
248,358,Protection du patrimoine bâti et des espaces p...,Histoire & Evolution de la ville,8ème période : XXIe,"Patrimoine Monumental, Patrimoine Paysager, Pa...",NaN,0.000000,0.000000


In [8]:
### 2nd way ###

new = patri_Grenoble.set_index(['Idtf','Titre','Périodes','Types','Adresse','Latitude',
                                'Longitude','Texte FR','Bibliographie',
                                'Crédit']).apply(lambda x: x.str.split(',')
                                .explode()).reset_index() 
new 

,Idtf,Titre,Périodes,Types,Adresse,Latitude,Longitude,Texte FR,Bibliographie,Crédit,Thématiques
0,14,Échauguette de l'ancienne citadelle,3ème période : 1590 / 1830,"Patrimoine Monumental (Militaire), Patrimoine ...","Quai Jongkind, Grenoble",45.194816,5.730774,"<h3>UN SYSTÈME DÉFENSIF, UNE NOUVELLE ENCEINTE...",NaN,NaN,Histoire & Evolution de la ville
1,14,Échauguette de l'ancienne citadelle,3ème période : 1590 / 1830,"Patrimoine Monumental (Militaire), Patrimoine ...","Quai Jongkind, Grenoble",45.194816,5.730774,"<h3>UN SYSTÈME DÉFENSIF, UNE NOUVELLE ENCEINTE...",NaN,NaN,Ville fortifiée/de garnison/militaire
2,20,"Place Victor-Hugo, cœur du nouveau centre-ville",5ème période : 1880 / 1925,Patrimoine Urbain (Place),"Place Victor Hugo, Grenoble",45.189111,5.724503,<p>UNE NOUVELLE URBANISATION ET SES ESPACES PU...,"- J-F. Parent, « Grenoble : deux siècles d'urb...",NaN,Histoire & Evolution de la ville
3,21,Enceinte romaine du 3e siècle,1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Enceinte / Rempart),Géolocalisation incomplète (un vestige parmi d...,45.192529,5.731154,LES TRACES D'UNE VILLE-GALLO ROMAINE Première ...,Office du Tourisme : www.grenoble-tourisme.com,NaN,Histoire & Evolution de la ville
4,21,Enceinte romaine du 3e siècle,1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Enceinte / Rempart),Géolocalisation incomplète (un vestige parmi d...,45.192529,5.731154,LES TRACES D'UNE VILLE-GALLO ROMAINE Première ...,Office du Tourisme : www.grenoble-tourisme.com,NaN,Ville fortifiée/de garnison/militaire
...,...,...,...,...,...,...,...,...,...,...,...
361,356,Dans les pas de Stendhal : les collections de ...,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","12, boulevard Maréchal-Lyautey, Grenoble",45.185253,5.731065,Historique de la constitution des collections ...,Mille ans d’écrits : trésors de la bibliothèqu...,Bibliothèque d'Etude et du Patrimoine et Ville...,Art et culture
362,356,Dans les pas de Stendhal : les collections de ...,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","12, boulevard Maréchal-Lyautey, Grenoble",45.185253,5.731065,Historique de la constitution des collections ...,Mille ans d’écrits : trésors de la bibliothèqu...,Bibliothèque d'Etude et du Patrimoine et Ville...,Tourisme
363,357,Église Saint-Pierre-du-Rondeau,7ème période : 1968 / aube XXIe,Patrimoine Monumental (Religieux),"Eglise Saint-Pierre-du-Rondeau, Grenoble",45.169375,5.711883,B. Avezou et P. Blondeau (architectes) Suite à...,"- J. Pichard (dir.), « Les églises du diocèse ...",Directions des Affaires Culturelles et de l'Ur...,Art et culture
364,358,Protection du patrimoine bâti et des espaces p...,8ème période : XXIe,"Patrimoine Monumental, Patrimoine Paysager, Pa...",NaN,0.000000,0.000000,La politique patrimoniale de la Ville de Greno...,NaN,Directions des Affaires Culturelles et de l'Ur...,Histoire & Evolution de la ville


In [9]:
new = new[['Idtf','Titre','Thématiques','Périodes','Types','Adresse','Latitude',
                                'Longitude','Texte FR','Bibliographie',
                                'Crédit']]
new

,Idtf,Titre,Thématiques,Périodes,Types,Adresse,Latitude,Longitude,Texte FR,Bibliographie,Crédit
0,14,Échauguette de l'ancienne citadelle,Histoire & Evolution de la ville,3ème période : 1590 / 1830,"Patrimoine Monumental (Militaire), Patrimoine ...","Quai Jongkind, Grenoble",45.194816,5.730774,"<h3>UN SYSTÈME DÉFENSIF, UNE NOUVELLE ENCEINTE...",NaN,NaN
1,14,Échauguette de l'ancienne citadelle,Ville fortifiée/de garnison/militaire,3ème période : 1590 / 1830,"Patrimoine Monumental (Militaire), Patrimoine ...","Quai Jongkind, Grenoble",45.194816,5.730774,"<h3>UN SYSTÈME DÉFENSIF, UNE NOUVELLE ENCEINTE...",NaN,NaN
2,20,"Place Victor-Hugo, cœur du nouveau centre-ville",Histoire & Evolution de la ville,5ème période : 1880 / 1925,Patrimoine Urbain (Place),"Place Victor Hugo, Grenoble",45.189111,5.724503,<p>UNE NOUVELLE URBANISATION ET SES ESPACES PU...,"- J-F. Parent, « Grenoble : deux siècles d'urb...",NaN
3,21,Enceinte romaine du 3e siècle,Histoire & Evolution de la ville,1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Enceinte / Rempart),Géolocalisation incomplète (un vestige parmi d...,45.192529,5.731154,LES TRACES D'UNE VILLE-GALLO ROMAINE Première ...,Office du Tourisme : www.grenoble-tourisme.com,NaN
4,21,Enceinte romaine du 3e siècle,Ville fortifiée/de garnison/militaire,1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Enceinte / Rempart),Géolocalisation incomplète (un vestige parmi d...,45.192529,5.731154,LES TRACES D'UNE VILLE-GALLO ROMAINE Première ...,Office du Tourisme : www.grenoble-tourisme.com,NaN
...,...,...,...,...,...,...,...,...,...,...,...
361,356,Dans les pas de Stendhal : les collections de ...,Art et culture,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","12, boulevard Maréchal-Lyautey, Grenoble",45.185253,5.731065,Historique de la constitution des collections ...,Mille ans d’écrits : trésors de la bibliothèqu...,Bibliothèque d'Etude et du Patrimoine et Ville...
362,356,Dans les pas de Stendhal : les collections de ...,Tourisme,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","12, boulevard Maréchal-Lyautey, Grenoble",45.185253,5.731065,Historique de la constitution des collections ...,Mille ans d’écrits : trésors de la bibliothèqu...,Bibliothèque d'Etude et du Patrimoine et Ville...
363,357,Église Saint-Pierre-du-Rondeau,Art et culture,7ème période : 1968 / aube XXIe,Patrimoine Monumental (Religieux),"Eglise Saint-Pierre-du-Rondeau, Grenoble",45.169375,5.711883,B. Avezou et P. Blondeau (architectes) Suite à...,"- J. Pichard (dir.), « Les églises du diocèse ...",Directions des Affaires Culturelles et de l'Ur...
364,358,Protection du patrimoine bâti et des espaces p...,Histoire & Evolution de la ville,8ème période : XXIe,"Patrimoine Monumental, Patrimoine Paysager, Pa...",NaN,0.000000,0.000000,La politique patrimoniale de la Ville de Greno...,NaN,Directions des Affaires Culturelles et de l'Ur...


In [10]:
new_pg = new[new['Adresse'].notnull()]  # ~ is opposite to == 

# comparing sizes of data frames 
print("Old data frame length:", len(new))
print("New data frame length:", len(new_pg))
print("Number of rows with at least 1 NA value: ", (len(new)-len(new_pg))) 

new_pg.notnull()

Old data frame length: 366
New data frame length: 294
Number of rows with at least 1 NA value:  72


,Idtf,Titre,Thématiques,Périodes,Types,Adresse,Latitude,Longitude,Texte FR,Bibliographie,Crédit
0,True,True,True,True,True,True,True,True,True,False,False
1,True,True,True,True,True,True,True,True,True,False,False
2,True,True,True,True,True,True,True,True,True,True,False
3,True,True,True,True,True,True,True,True,True,True,False
4,True,True,True,True,True,True,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...
360,True,True,True,True,True,True,True,True,True,False,True
361,True,True,True,True,True,True,True,True,True,True,True
362,True,True,True,True,True,True,True,True,True,True,True
363,True,True,True,True,True,True,True,True,True,True,True


In [11]:
mask = (new_pg['Latitude'] > 0.0001) & (new_pg['Longitude'] > 0.0001)

new_pg = new_pg[mask]

new_pg

,Idtf,Titre,Thématiques,Périodes,Types,Adresse,Latitude,Longitude,Texte FR,Bibliographie,Crédit
0,14,Échauguette de l'ancienne citadelle,Histoire & Evolution de la ville,3ème période : 1590 / 1830,"Patrimoine Monumental (Militaire), Patrimoine ...","Quai Jongkind, Grenoble",45.194816,5.730774,"<h3>UN SYSTÈME DÉFENSIF, UNE NOUVELLE ENCEINTE...",NaN,NaN
1,14,Échauguette de l'ancienne citadelle,Ville fortifiée/de garnison/militaire,3ème période : 1590 / 1830,"Patrimoine Monumental (Militaire), Patrimoine ...","Quai Jongkind, Grenoble",45.194816,5.730774,"<h3>UN SYSTÈME DÉFENSIF, UNE NOUVELLE ENCEINTE...",NaN,NaN
2,20,"Place Victor-Hugo, cœur du nouveau centre-ville",Histoire & Evolution de la ville,5ème période : 1880 / 1925,Patrimoine Urbain (Place),"Place Victor Hugo, Grenoble",45.189111,5.724503,<p>UNE NOUVELLE URBANISATION ET SES ESPACES PU...,"- J-F. Parent, « Grenoble : deux siècles d'urb...",NaN
3,21,Enceinte romaine du 3e siècle,Histoire & Evolution de la ville,1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Enceinte / Rempart),Géolocalisation incomplète (un vestige parmi d...,45.192529,5.731154,LES TRACES D'UNE VILLE-GALLO ROMAINE Première ...,Office du Tourisme : www.grenoble-tourisme.com,NaN
4,21,Enceinte romaine du 3e siècle,Ville fortifiée/de garnison/militaire,1ére période : -43 / fin VIIe début VIIIe,Patrimoine Urbain (Enceinte / Rempart),Géolocalisation incomplète (un vestige parmi d...,45.192529,5.731154,LES TRACES D'UNE VILLE-GALLO ROMAINE Première ...,Office du Tourisme : www.grenoble-tourisme.com,NaN
...,...,...,...,...,...,...,...,...,...,...,...
360,353,Dans les pas de Stendhal : l’hôtel de Lesdigui...,Art et culture,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","Passage du Palais de Justice, Grenoble\r\n",45.192750,5.727186,L'Hôtel de Lesdiguières et son jardin attenant...,NaN,Bibliothèque d'Etude et du Patrimoine et Ville...
361,356,Dans les pas de Stendhal : les collections de ...,Art et culture,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","12, boulevard Maréchal-Lyautey, Grenoble",45.185253,5.731065,Historique de la constitution des collections ...,Mille ans d’écrits : trésors de la bibliothèqu...,Bibliothèque d'Etude et du Patrimoine et Ville...
362,356,Dans les pas de Stendhal : les collections de ...,Tourisme,3ème période : 1590 / 1830,"Patrimoine Immatériel, Patrimoine Monumental (...","12, boulevard Maréchal-Lyautey, Grenoble",45.185253,5.731065,Historique de la constitution des collections ...,Mille ans d’écrits : trésors de la bibliothèqu...,Bibliothèque d'Etude et du Patrimoine et Ville...
363,357,Église Saint-Pierre-du-Rondeau,Art et culture,7ème période : 1968 / aube XXIe,Patrimoine Monumental (Religieux),"Eglise Saint-Pierre-du-Rondeau, Grenoble",45.169375,5.711883,B. Avezou et P. Blondeau (architectes) Suite à...,"- J. Pichard (dir.), « Les églises du diocèse ...",Directions des Affaires Culturelles et de l'Ur...


In [12]:
new_pg.Thématiques = new_pg.Thématiques.str.lstrip()  #delete spaces in left of values

In [13]:
# Replace old values with new values

new_pg.Thématiques = new_pg.Thématiques.str.replace('Ville fortifiée/de garnison/militaire',
                                                    "Ville fortifiée de garnison et militaire")
new_pg.Thématiques = new_pg.Thématiques.str.replace('Sciences/techniques/innovation',
                                                    "Sciences, techniques et innovation")

In [14]:
new_pg.isnull().sum()

Idtf               0
Titre              0
Thématiques        0
Périodes           0
Types              0
Adresse            0
Latitude           0
Longitude          0
Texte FR           0
Bibliographie     64
Crédit           170
dtype: int64

In [15]:
new_pg.describe()

,Idtf,Latitude,Longitude
count,294.000000,294.000000,294.000000
mean,162.479592,45.189446,5.728599
std,97.820470,0.015087,0.013884
min,14.000000,45.129601,5.588144
25%,81.250000,45.185580,5.725136
50%,152.500000,45.190420,5.730774
75%,240.500000,45.193057,5.733604
max,359.000000,45.339375,5.812808


In [16]:
# Last Step
if conn is not None:
    new_pg.to_sql('patrimoine_Grenoble',conn, if_exists = 'replace')

cur.execute("SELECT * FROM patrimoine_Grenoble")
result = cur.fetchall()

print(len(result))
print(result)

294
[(0, 14, "Échauguette de l'ancienne citadelle", 'Histoire & Evolution de la ville', '3ème période : 1590 / 1830', 'Patrimoine Monumental (Militaire), Patrimoine Urbain (Enceinte / Rempart)', 'Quai Jongkind, Grenoble', 45.194815729169406, 5.7307737405838, '<h3>UN SYSTÈME DÉFENSIF, UNE NOUVELLE ENCEINTE</h3> <p>Cette échauguette faisait partie d’un ensemble fortifié, la citadelle, créé en 1592, deux ans après la prise de Grenoble par le duc de Lesdiguières. Située au nord-ouest de l’enceinte, en surplomb sur l’Isère, cette architecture militaire classique abrite les sentinelles qui surveillent l’extérieur. La citadelle comprenait également la tour de l’Île, plus ancienne (14e siècle), transformée en arsenal et en logement pour le gouverneur. En 1860, elle est transformée en caserne (Vinoy), avant d’être désaffectée en 1959, puis démolie lors des Jeux olympiques de 1968. Seuls la tour de l’Île, l’échauguette et les remparts la soutenant seront conservés. L’espace libéré est alors util

/home/zahra/.local/share/virtualenvs/projet-patrimoine-de-grenoble-OckOP8At/lib/python3.8/site-packages/pandas/core/generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [17]:
import folium
from folium import FeatureGroup, LayerControl, plugins
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import requests

ModuleNotFoundError: No module named 'folium'

In [21]:
Lat = new_pg['Latitude'].mean() 
Lng = new_pg['Longitude'].mean() 

Grenoble = folium.Map(location=[Lat,Lng], 
                        zoom_start=11) 

#color argument of Icon should be one of: {'orange', 'blue', 'gray', 'green', 'cadetblue', 'darkred',
#'lightblue', 'black', 'red', 'darkgreen', 'darkpurple', 'beige', 'lightred', 'pink', 'lightgreen',
#'purple', 'white', 'darkblue', 'lightgray'}

def color(thematic):
    for them in thematic:
        if thematic == 'Art et culture':
            col = 'green'
        elif thematic == 'Histoire & Evolution de la ville':
            col = 'blue'
        elif thematic == 'Patrimoine naturel et paysager':
            col = 'orange'
        elif thematic == 'Sciences, techniques et innovation':
            col = 'purple'
        elif thematic == 'Tourisme':
            col = 'beige'
        elif thematic == 'Ville fortifiée de garnison et militaire':
            col='red'
        else:
            col = 'gray'
    return col 
      
for lat,lan,name in zip(new_pg['Latitude'],new_pg['Longitude'],new_pg['Thématiques']):  

    folium.Marker(location = [lat,lan], popup = name, 
                  icon = folium.Icon(color = color(name), 
                  icon_color='white')).add_to(Grenoble)
    #print(name,color(name))  
    folium.map.LayerControl()
Grenoble

In [22]:
latmean = new_pg['Latitude'].mean() 
lonmean = new_pg['Longitude'].mean()

Grenoble = folium.Map(
            location=[latmean,lonmean], 
            zoom_start=11,
            tiles='Stamen Terrain'
            #attr='Mapbox attribution'
)

popup = folium.Popup(new_pg['Thématiques'])

#fg = folium.FeatureGroup(name = 'Thématiques')
g1 = folium.FeatureGroup(name = 'Art et culture')
g2 = folium.FeatureGroup(name = 'Histoire & évolution de la ville')
g3 = folium.FeatureGroup(name = 'Patrimoine naturel et paysager')
g4 = folium.FeatureGroup(name = 'Sciences, techniques et innovations')
g5 = folium.FeatureGroup(name = 'Tourisme')
g6 = folium.FeatureGroup(name = 'Ville fortifiée de garnison ou militaire')
g7 = folium.FeatureGroup(name = 'Others')

#Grenoble.add_child(fg)
Grenoble.add_child(g1)   
Grenoble.add_child(g2)
Grenoble.add_child(g3)
Grenoble.add_child(g4)   
Grenoble.add_child(g5)
Grenoble.add_child(g6)
Grenoble.add_child(g7)

def color(thematic):
    for them in thematic:
        if thematic == 'Art et culture':
            col = 'green'
        elif thematic == 'Histoire & Evolution de la ville':
            col = 'blue'
        elif thematic == 'Patrimoine naturel et paysager':
            col = 'orange'
        elif thematic == 'Sciences, techniques et innovation':
            col = 'purple'
        elif thematic == 'Tourisme':
            col = 'beige'
        elif thematic == 'Ville fortifiée de garnison et militaire':
            col='red'
        else:
            col = 'gray'
    return col

for lat,lan,them in zip(new_pg['Latitude'],new_pg['Longitude'],new_pg['Thématiques']):
    if them == 'Art et culture':
        g1.add_child(folium.Marker([lat,lan],icon=folium.Icon(color=color(them)),popup=them))
    elif them == 'Histoire & Evolution de la ville':
        g2.add_child(folium.Marker([lat,lan],icon=folium.Icon(color=color(them)),popup=them))
    elif them == 'Patrimoine naturel et paysager':
        g3.add_child(folium.Marker([lat,lan],icon=folium.Icon(color=color(them)),popup=them))
    elif them == 'Sciences, techniques et innovation':
        g4.add_child(folium.Marker([lat,lan],icon=folium.Icon(color=color(them)),popup=them))
    elif them == 'Tourisme':
        g5.add_child(folium.Marker([lat,lan],icon=folium.Icon(color=color(them)),popup=them))
    elif them == 'Ville fortifiée de garnison et militaire':
        g6.add_child(folium.Marker([lat,lan],icon=folium.Icon(color=color(them)),popup=them))
    else:
        g7.add_child(folium.Marker([lat,lan],icon=folium.Icon(color=color(them)),popup=them))

folium.LayerControl(position='topleft').add_to(Grenoble)

Grenoble